## Preparação dos Dados para Treinamento da YOLO

### Objetivo
Este notebook tem como objetivo formatar os dados de imagens de mergulhadores fazendo gestos para o formato aceito pela YOLOv8. Especificamente, estamos transformando o CSV de `true negatives` (imagens sem gestos) para que tenha o mesmo formato do CSV de `true positives` (imagens com gestos).

### Passos Executados
1. **Carregamento dos CSVs:**
   - `true_positives.csv`: Contém as imagens com gestos e suas respectivas anotações.
   - `true_negatives.csv`: Contém as imagens sem gestos.

2. **Visualização Inicial:**
   - Exibição das primeiras linhas de cada CSV para entender sua estrutura.

3. **Adição de Colunas Faltantes em `true_negatives`:**
   - Colunas adicionadas: `roi left`, `roi right`, `iqa_mdm_entropy`, `iqa_mdm_d`, `iqa_mdm_dcomp`, `param 1`, `param 2`.
   - Valores padrão atribuídos:
     - `roi left` e `roi right`: `[0,0,0,0]` (indicando ausência de ROI).
     - `iqa_mdm_entropy`, `iqa_mdm_d`, `iqa_mdm_dcomp`: `0.0`.
     - `param 1` e `param 2`: Strings vazias.

4. **Atualização dos Labels:**
   - `label name`: Definido como "Negative".
   - `label id`: Definido como `16`.

5. **Renomeação de Colunas:**
   - Ajuste de nomes de colunas para corresponder ao formato do `true_positives`.

6. **Reorganização das Colunas:**
   - As colunas do `true_negatives` foram reorganizadas para corresponder à ordem do `true_positives`.

7. **Salvar o Novo CSV:**
   - O CSV formatado de `true negatives` foi salvo como `true_negatives_formatted.csv`.



In [2]:
import pandas as pd

# Carregar os CSVs de true positives e true negatives
true_positives = pd.read_csv('Dataset_original/CADDY_gestures_all_true_positives_release_v2.csv')
true_negatives = pd.read_csv('Dataset_original/CADDY_gestures_all_true_negatives_release_v2.csv')

# Visualizar as primeiras linhas dos DataFrames para entender a estrutura
print("True Positives:")
print(true_positives.head())
print("\nTrue Negatives:")
print(true_negatives.head())

# Adicionar colunas faltantes no true_negatives
true_negatives['roi left'] = ''
true_negatives['roi right'] = ''
true_negatives['iqa_mdm_entropy'] = ''
true_negatives['iqa_mdm_d'] = ''
true_negatives['iqa_mdm_dcomp'] = ''
true_negatives['param 1'] = ''
true_negatives['param 2'] = ''

# Renomear as colunas do true_negatives para corresponder ao true_positives
true_negatives = true_negatives.rename(columns={
    'selected img': '',
    'distortion type': 'distortion',
    'dist. param 1': 'param 1',
    'dist. param 2': 'param 2'
})

# Reorganizar as colunas para corresponder à ordem do true_positives
true_negatives = true_negatives[true_positives.columns]

# Salvar o novo CSV de true negatives formatado
true_negatives.to_csv('Dataset_original/true_negatives_formatted.csv', index=False)

print("True Negatives Formatted:")
print(true_negatives.head())


True Positives:
   index   scenario                                        stereo left   
0      0  biograd-A  /biograd-A/true_positives/raw/biograd-A_00000_...  \
1      1  biograd-A  /biograd-A/true_positives/blurred/dir_00/biogr...   
2      2  biograd-A  /biograd-A/true_positives/blurred/dir_01/biogr...   
3      3  biograd-A  /biograd-A/true_positives/blurred/dir_02/biogr...   
4      4  biograd-A  /biograd-A/true_positives/noisy/dir_00/biograd...   

                                        stereo right     label name  label id   
0  /biograd-A/true_positives/raw/biograd-A_00000_...  num_delimiter        10  \
1  /biograd-A/true_positives/blurred/dir_00/biogr...  num_delimiter        10   
2  /biograd-A/true_positives/blurred/dir_01/biogr...  num_delimiter        10   
3  /biograd-A/true_positives/blurred/dir_02/biogr...  num_delimiter        10   
4  /biograd-A/true_positives/noisy/dir_00/biograd...  num_delimiter        10   

          roi left        roi right  synthetic  iqa_


Para resolver esse problema, podemos criar um código que renomeia os arquivos na pasta de true negatives, adicionando o prefixo "neg_" aos nomes dos arquivos. Em seguida, atualizaremos o CSV de true negatives para refletir esses novos nomes de arquivos.

Aqui está o código para renomear os arquivos e atualizar o CSV:

python
Copiar código
import os
import pandas as pd

# Caminho para a pasta de true negatives
negatives_folder = 'Dataset/true_negatives'

# Renomear os arquivos na pasta de true negatives
for filename in os.listdir(negatives_folder):
    if filename.endswith('.jpg'):
        new_filename = 'neg_' + filename
        os.rename(os.path.join(negatives_folder, filename), os.path.join(negatives_folder, new_filename))

# Carregar o CSV de true negatives
true_negatives = pd.read_csv('Dataset/CADDY_gestures_all_true_negatives_release_v2.csv')

# Adicionar o prefixo "neg_" aos nomes dos arquivos no CSV
true_negatives['stereo left'] = true_negatives['stereo left'].apply(lambda x: os.path.join('Dataset/true_negatives', 'neg_' + os.path.basename(x)))
true_negatives['stereo right'] = true_negatives['stereo right'].apply(lambda x: os.path.join('Dataset/true_negatives', 'neg_' + os.path.basename(x)))

# Adicionar colunas faltantes no true_negatives com valores padrão
true_negatives['roi left'] = '[0,0,0,0]'
true_negatives['roi right'] = '[0,0,0,0]'
true_negatives['iqa_mdm_entropy'] = 0.0
true_negatives['iqa_mdm_d'] = 0.0
true_negatives['iqa_mdm_dcomp'] = 0.0
true_negatives['param 1'] = ''
true_negatives['param 2'] = ''
true_negatives['label name'] = 'Negative'
true_negatives['label id'] = 16

# Renomear as colunas do true_negatives para corresponder ao true_positives
true_negatives = true_negatives.rename(columns={
    'selected img': '',
    'distortion type': 'distortion',
    'dist. param 1': 'param 1',
    'dist. param 2': 'param 2'
})

# Carregar o CSV de true positives para obter a ordem correta das colunas
true_positives = pd.read_csv('Dataset/CADDY_gestures_all_true_positives_release_v2.csv')

# Reorganizar as colunas para corresponder à ordem do true_positives
true_negatives = true_negatives[true_positives.columns]

# Salvar o novo CSV de true negatives formatado
true_negatives.to_csv('Dataset/true_negatives_formatted.csv', index=False)

print("True Negatives Formatted:")
print(true_negatives.head())
Resumo em Markdown
markdown
Copiar código
## Renomeação de Arquivos e Atualização do CSV

### Objetivo
Este notebook tem como objetivo renomear os arquivos de imagens na pasta de `true negatives` adicionando o prefixo "neg_" e atualizar o CSV correspondente para refletir essas mudanças.

### Passos Executados
1. **Renomeação de Arquivos:**
   - Adição do prefixo "neg_" aos nomes dos arquivos de imagens na pasta `true negatives`.

2. **Atualização do CSV de True Negatives:**
   - Atualização dos caminhos das imagens no CSV de true negatives para incluir o prefixo "neg_".
   - Adição de colunas faltantes (`roi left`, `roi right`, `iqa_mdm_entropy`, `iqa_mdm_d`, `iqa_mdm_dcomp`, `param 1`, `param 2`) com valores padrão.
   - Definição de `label name` como "Negative" e `label id` como `16`.

3. **Renomeação de Colunas:**
   - Ajuste de nomes de colunas para corresponder ao formato do `true_positives`.

4. **Reorganização das Colunas:**
   - As colunas do `true_negatives` foram reorganizadas para corresponder à ordem do `true_positives`.

5. **Salvar o Novo CSV:**
   - O CSV formatado de `true negatives` foi salvo como `true_negatives_formatted.csv`.

In [4]:
import os
import pandas as pd

# Caminho para a pasta de true negatives
negatives_folder = 'Dataset_original/true_negative'

# Renomear os arquivos na pasta de true negatives
for filename in os.listdir(negatives_folder):
    if filename.endswith('.jpg'):
        new_filename = 'neg_' + filename
        os.rename(os.path.join(negatives_folder, filename), os.path.join(negatives_folder, new_filename))

# Carregar o CSV de true negatives
true_negatives = pd.read_csv('Dataset_original/CADDY_gestures_all_true_negatives_release_v2.csv')

# Adicionar o prefixo "neg_" aos nomes dos arquivos no CSV
true_negatives['stereo left'] = true_negatives['stereo left'].apply(lambda x: os.path.join('Dataset/true_negatives', 'neg_' + os.path.basename(x)))
true_negatives['stereo right'] = true_negatives['stereo right'].apply(lambda x: os.path.join('Dataset/true_negatives', 'neg_' + os.path.basename(x)))

# Adicionar colunas faltantes no true_negatives com valores padrão
true_negatives['roi left'] = '[0,0,0,0]'
true_negatives['roi right'] = '[0,0,0,0]'
true_negatives['iqa_mdm_entropy'] = 0.0
true_negatives['iqa_mdm_d'] = 0.0
true_negatives['iqa_mdm_dcomp'] = 0.0
true_negatives['param 1'] = ''
true_negatives['param 2'] = ''
true_negatives['label name'] = 'Negative'
true_negatives['label id'] = 16

# Renomear as colunas do true_negatives para corresponder ao true_positives
true_negatives = true_negatives.rename(columns={
    'selected img': '',
    'distortion type': 'distortion',
    'dist. param 1': 'param 1',
    'dist. param 2': 'param 2'
})

# Carregar o CSV de true positives para obter a ordem correta das colunas
true_positives = pd.read_csv('Dataset_original/CADDY_gestures_all_true_positives_release_v2.csv')

# Reorganizar as colunas para corresponder à ordem do true_positives
true_negatives = true_negatives[true_positives.columns]

# Salvar o novo CSV de true negatives formatado
true_negatives.to_csv('Dataset_original/true_negatives_formatted.csv', index=False)

print("True Negatives Formatted:")
print(true_negatives.head())


True Negatives Formatted:
   index   scenario                                        stereo left   
0      0  biograd-A  Dataset/true_negatives\neg_biograd-A_00000_lef...  \
1      1  biograd-A  Dataset/true_negatives\neg_biograd-A_00000_lef...   
2      2  biograd-A  Dataset/true_negatives\neg_biograd-A_00000_lef...   
3      3  biograd-A  Dataset/true_negatives\neg_biograd-A_00000_lef...   
4      4  biograd-A  Dataset/true_negatives\neg_biograd-A_00000_lef...   

                                        stereo right label name  label id   
0  Dataset/true_negatives\neg_biograd-A_00000_rig...   Negative        16  \
1  Dataset/true_negatives\neg_biograd-A_00000_rig...   Negative        16   
2  Dataset/true_negatives\neg_biograd-A_00000_rig...   Negative        16   
3  Dataset/true_negatives\neg_biograd-A_00000_rig...   Negative        16   
4  Dataset/true_negatives\neg_biograd-A_00000_rig...   Negative        16   

    roi left  roi right  synthetic  iqa_mdm_entropy  iqa_mdm_d  iq

## Combinação dos CSVs de True Positives e True Negatives

### Objetivo
Este notebook tem como objetivo combinar os CSVs de `true positives` (original) e `true negatives` (alterado) em um único CSV, com os dados de true negatives sendo adicionados abaixo dos dados de true positives.




In [6]:
import pandas as pd

# Carregar o CSV de true positives (original) e o CSV de true negatives (alterado)
true_positives = pd.read_csv('Dataset_original/CADDY_gestures_all_true_positives_release_v2.csv')
true_negatives = pd.read_csv('Dataset_original/true_negatives_formatted.csv')

# Concatenar os DataFrames
combined_df = pd.concat([true_positives, true_negatives], ignore_index=True)

# Salvar o novo CSV combinado
combined_df.to_csv('Dataset_original/combined_gestures.csv', index=False)

print("Combined DataFrame:")
print(combined_df.head())
print(combined_df.tail())



num_items = combined_df.shape[0]

print(f"O arquivo combined_gestures.csv contém {num_items} itens.")


Combined DataFrame:
   index   scenario                                        stereo left   
0      0  biograd-A  /biograd-A/true_positives/raw/biograd-A_00000_...  \
1      1  biograd-A  /biograd-A/true_positives/blurred/dir_00/biogr...   
2      2  biograd-A  /biograd-A/true_positives/blurred/dir_01/biogr...   
3      3  biograd-A  /biograd-A/true_positives/blurred/dir_02/biogr...   
4      4  biograd-A  /biograd-A/true_positives/noisy/dir_00/biograd...   

                                        stereo right     label name  label id   
0  /biograd-A/true_positives/raw/biograd-A_00000_...  num_delimiter        10  \
1  /biograd-A/true_positives/blurred/dir_00/biogr...  num_delimiter        10   
2  /biograd-A/true_positives/blurred/dir_01/biogr...  num_delimiter        10   
3  /biograd-A/true_positives/blurred/dir_02/biogr...  num_delimiter        10   
4  /biograd-A/true_positives/noisy/dir_00/biograd...  num_delimiter        10   

          roi left        roi right  synthetic  

## Considerações sobre Manter ou Remover Imagens "Left"

### Objetivo
Este notebook tem como objetivo discutir as vantagens e desvantagens de manter ou remover as imagens "left" de um dataset de imagens estéreo, e fornecer o código necessário para remover essas imagens, se desejado.

### Considerações
1. **Manter Ambas as Imagens:**
   - Aumenta o número de dados disponíveis.
   - Pode fornecer diferentes perspectivas.
   - Pode capturar informações mais ricas.

2. **Remover Imagens Duplicadas:**
   - Pode reduzir o risco de overfitting.
   - Reduz o custo computacional.

### Recomendações
- Experimente ambas as abordagens e compare o desempenho.
- Use validação cruzada para avaliar o risco de overfitting.
- Use aumento de dados (data augmentation) se remover uma das cópias.

In [8]:
import pandas as pd
import os

# Carregar o CSV combinado
combined_df = pd.read_csv('Dataset_original/combined_gestures.csv')

# Filtrar apenas as imagens "right"
combined_df_right = combined_df[combined_df['stereo right'].notna()]

# Salvar o novo CSV apenas com imagens "right"
combined_df_right.to_csv('Dataset_original/combined_gestures_right.csv', index=False)

print(f"O novo CSV contém {combined_df_right.shape[0]} linhas.")

# Caminho para a pasta de imagens
images_folder = 'Dataset_original/images'

# Remover imagens "left" da pasta
for filename in os.listdir(images_folder):
    if '_left.jpg' in filename:
        os.remove(os.path.join(images_folder, filename))

print("Imagens 'left' removidas.")


O novo CSV contém 164290 linhas.
Imagens 'left' removidas.


## Conversão de Anotações para Formato YOLOv8

### Objetivo
Este notebook tem como objetivo converter o dataset de anotações para o formato compatível com YOLOv8, utilizando as classes e ROIs fornecidas no CSV.

### Passos Executados
1. **Definição da Função de Conversão:**
   - Função `convert_to_yolo` para converter as coordenadas de ROI para o formato YOLO.

2. **Carregamento do CSV:**
   - Carregamento do arquivo `combined_gestures_right.csv`.

3. **Criação da Pasta de Saída:**
   - Criação da pasta `annotations_yolo` para armazenar as anotações no formato YOLO.

4. **Processamento das Linhas do CSV:**
   - Para cada linha no CSV, extração das informações de caminho da imagem, ID da classe e ROI.
   - Conversão das ROIs para o formato YOLO.
   - Criação dos arquivos de anotação correspondentes para cada imagem.

In [10]:
import pandas as pd
import os
import ast

# Função para converter coordenadas de ROI para o formato YOLO
def convert_to_yolo(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2] / 2.0) * dw
    y = (box[1] + box[3] / 2.0) * dh
    w = box[2] * dw
    h = box[3] * dh
    return (x, y, w, h)

# Carregar o CSV combinado
csv_file = 'Dataset_original/combined_gestures_right.csv'
df = pd.read_csv(csv_file)

# Pasta de saída para as anotações no formato YOLO
output_folder = 'Dataset_original/annotations_yolo'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Tamanho das imagens (assumindo 640x480, ajuste conforme necessário)
image_width = 640
image_height = 480

# Processar cada linha do CSV
for index, row in df.iterrows():
    try:
        # Extrair informações do CSV
        image_path = row['stereo right']
        class_id = row['label id']
        roi_str = row['roi right']
        
        # Converter string para lista, garantindo que seja uma lista válida
        roi = ast.literal_eval(roi_str)
        if not isinstance(roi, list) or len(roi) != 4:
            raise ValueError("ROI is not a valid list")
        
        # Converter ROI para formato YOLO
        yolo_box = convert_to_yolo((image_width, image_height), roi)
        
        # Nome do arquivo de anotação
        annotation_file = os.path.join(output_folder, os.path.basename(image_path).replace('.jpg', '.txt'))
        
        # Escrever a anotação no arquivo
        with open(annotation_file, 'w') as f:
            f.write(f"{class_id} {yolo_box[0]} {yolo_box[1]} {yolo_box[2]} {yolo_box[3]}\n")
    
    except (SyntaxError, ValueError) as e:
        print(f"Erro ao processar a linha {index}: {e}")

print("Conversão concluída!")


Erro ao processar a linha 6890: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6891: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6892: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6893: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6894: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6895: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6896: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6897: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6898: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6899: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6900: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6901: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6902: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6903: invalid syntax (<unknown>, line 1)
Erro ao processar a linha 6904: invalid syntax (<unknown>, lin

## Validação e Cópia de Imagens Baseada em Anotações YOLO

### Objetivo
Este notebook tem como objetivo validar e copiar apenas as imagens que possuem anotações correspondentes na pasta de anotações YOLO, criando uma nova pasta `images_validated` com essas imagens.

### Passos Executados
1. **Obtenção de Nomes de Imagens Válidas:**
   - Extração dos nomes das imagens válidas a partir dos arquivos de anotações YOLO.

2. **Criação da Pasta de Imagens Validadas:**
   - Criação da pasta `images_validated` para armazenar as imagens validadas.

3. **Cópia das Imagens Válidas:**
   - Cópia das imagens válidas da pasta original para a nova pasta.

In [11]:
import os
import shutil

# Caminho para a pasta de imagens original e a nova pasta de imagens validadas
images_folder = 'Dataset_original/images'
validated_images_folder = 'Dataset_original/images_validated'

# Caminho para a pasta de anotações YOLO
annotations_folder = 'Dataset_original/annotations_yolo'

# Obter a lista de nomes de imagens válidas a partir das anotações YOLO
valid_image_names = set()
for annotation_file in os.listdir(annotations_folder):
    if annotation_file.endswith('.txt'):
        image_name = annotation_file.replace('.txt', '.jpg')
        valid_image_names.add(image_name)

# Criar a pasta de imagens validadas se não existir
if not os.path.exists(validated_images_folder):
    os.makedirs(validated_images_folder)

# Copiar apenas as imagens válidas para a nova pasta
for filename in os.listdir(images_folder):
    if filename in valid_image_names:
        shutil.copy(os.path.join(images_folder, filename), os.path.join(validated_images_folder, filename))

print("Imagens validadas copiadas para a pasta 'images_validated'.")


Imagens validadas copiadas para a pasta 'images_validated'.


## Preparação do Dataset para YOLOv8

### Objetivo
Este notebook tem como objetivo organizar o dataset de imagens e anotações em uma estrutura compatível com YOLOv8, criando conjuntos de treino, validação e teste, além de gerar o arquivo de configuração `dataset.yaml`.

### Passos Executados
1. **Instalação da Biblioteca `scikit-learn`:**
   - Execute o comando `pip install scikit-learn` no terminal para instalar a biblioteca necessária.

2. **Divisão dos Conjuntos de Treino, Validação e Teste:**
   - As imagens e anotações foram divididas em 70% para treino, 20% para validação e 10% para teste.

3. **Criação das Pastas de Saída:**
   - Foram criadas as pastas `images/train`, `images/val`, `images/test`, `labels/train`, `labels/val` e `labels/test`.

4. **Movimentação de Arquivos:**
   - As imagens e anotações foram movidas para as pastas de treino, validação e teste correspondentes.

5. **Geração do Arquivo dataset.yaml:**
   - Um arquivo `dataset.yaml` foi gerado para especificar a estrutura do dataset e as classes.


In [15]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Caminhos das pastas originais e das pastas de destino
images_folder = 'Dataset_original/images_validated'
annotations_folder = 'Dataset_original/annotations_yolo'
output_base = 'Dataset'

# Definir caminhos de saída
images_train_folder = os.path.join(output_base, 'images', 'train')
images_val_folder = os.path.join(output_base, 'images', 'val')
images_test_folder = os.path.join(output_base, 'images', 'test')
labels_train_folder = os.path.join(output_base, 'labels', 'train')
labels_val_folder = os.path.join(output_base, 'labels', 'val')
labels_test_folder = os.path.join(output_base, 'labels', 'test')

# Criar diretórios de saída, se não existirem
os.makedirs(images_train_folder, exist_ok=True)
os.makedirs(images_val_folder, exist_ok=True)
os.makedirs(images_test_folder, exist_ok=True)
os.makedirs(labels_train_folder, exist_ok=True)
os.makedirs(labels_val_folder, exist_ok=True)
os.makedirs(labels_test_folder, exist_ok=True)

# Obter lista de todas as imagens validadas
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]

# Dividir as imagens em conjuntos de treino, validação e teste
train_files, test_files = train_test_split(image_files, test_size=0.1, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)  # 20% do 90% restante para validação

# Função para mover arquivos
def move_files(file_list, source_folder, dest_folder):
    for file_name in file_list:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(dest_folder, file_name))

# Mover arquivos de treino
move_files(train_files, images_folder, images_train_folder)
move_files([f.replace('.jpg', '.txt') for f in train_files], annotations_folder, labels_train_folder)

# Mover arquivos de validação
move_files(val_files, images_folder, images_val_folder)
move_files([f.replace('.jpg', '.txt') for f in val_files], annotations_folder, labels_val_folder)

# Mover arquivos de teste
move_files(test_files, images_folder, images_test_folder)
move_files([f.replace('.jpg', '.txt') for f in test_files], annotations_folder, labels_test_folder)

print("Imagens e anotações movidas para as pastas de treino, validação e teste.")

# Criar arquivo de configuração dataset.yaml
dataset_yaml_content = f"""
path: {os.path.abspath(output_base)}  # caminho para o dataset
train: images/train  # caminho para as imagens de treino
val: images/val  # caminho para as imagens de validação
test: images/test  # caminho para as imagens de teste

# Classes
names:
  0: start_comm
  1: end_comm
  2: up
  3: down
  4: photo
  5: backwards
  6: carry
  7: boat
  8: here
  9: mosaic
  10: num_delimiter
  11: one
  12: two
  13: three
  14: four
  15: five
  16: negative
"""

with open(os.path.join(output_base, 'dataset.yaml'), 'w') as f:
    f.write(dataset_yaml_content)

print("Arquivo dataset.yaml criado.")


Imagens e anotações movidas para as pastas de treino, validação e teste.
Arquivo dataset.yaml criado.
